In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama

prompt = PromptTemplate.from_template(
      """
      <s> [INST] 
      Return true if given sentence is a question or inquiry or a statement that induces response. This includes statement that expresses confusion or ambiguity. Return false otherwise.
      Do not try to explain, just simply provide true or false answer.
      [/INST] </s> 
      [INST]
      Sentence: {sentence} 
      Answer: 
      [/INST]
      """
)


llm = ChatOllama(model="llama3.1:latest", temperature=0)
chain = prompt | llm

sentence1 = '''Thank you so much man, this is awesome content''' # False
sentence2 = '''Can you make another video like this.''' # True
sentence3 = '''Can I ask you this, make another video like this!''' # True
sentence4 = '''What a beautiful video.''' # False
sentence5 = '''What were you saying at 5:13 mark''' # True 
sentence6 = '''Where can I find more material like this''' # True
sentence7 = '''I don't get when you said this''' # True
sentence8 = '''What were you saying at 5:13 mark''' # True 
sentence9 = '''I don't understand this part that says''' # True
sentence10 = '''I really don't get why people are saying bad things about this video''' # True




In [2]:
query_chain = chain.invoke(
    {
        "sentence": sentence1
    }
)
print(query_chain.content)

expected = False
print(expected)

False
False


In [3]:
query_chain = chain.invoke(
    {
        "sentence": sentence2
    }
)
print(query_chain.content)

expected = True
print(expected)

True.
True


In [4]:
query_chain = chain.invoke(
    {
        "sentence": sentence3
    }
)
print(query_chain.content)

expected = True
print(expected)

True
True


In [5]:
query_chain = chain.invoke(
    {
        "sentence": sentence4
    }
)
print(query_chain.content)

expected = False
print(expected)

False
False


In [6]:
query_chain = chain.invoke(
    {
        "sentence": sentence5
    }
)
print(query_chain.content)

expected = True
print(expected)

True
True


In [7]:
query_chain = chain.invoke(
    {
        "sentence": sentence6
    }
)
print(query_chain.content)

expected = True
print(expected)

True
True


In [8]:
query_chain = chain.invoke(
    {
        "sentence": sentence7
    }
)
print(query_chain.content)

expected = True
print(expected)

True
True


In [9]:
query_chain = chain.invoke(
    {
        "sentence": sentence8
    }
)
print(query_chain.content)

expected = True
print(expected)

True
True


In [10]:
query_chain = chain.invoke(
    {
        "sentence": sentence9
    }
)
print(query_chain.content)

expected = True
print(expected)

True
True


In [11]:
query_chain = chain.invoke(
    {
        "sentence": sentence10
    }
)
print(query_chain.content)

expected = False
print(expected)

True.
False
